In [10]:
# pip install sentence-transformers 
import os
import pandas as pd
from sentence_transformers import SentenceTransformer  # 한국어 처리 모델
from sklearn.metrics.pairwise import cosine_similarity  # 유사도 분석

# 데이터셋 로드

원천 데이터 (.txt)

In [34]:
import zipfile
# 추출할 zip 파일 경로
for i in range(1,5):
    zip_file_path = f"C:/dev/github/pythonbasic/mental-health-chatbot/data/Training/TS_01. KAKAO({i}).zip"
    # 추출할 위치
    extract_path = "C:/dev/github/pythonbasic/mental-health-chatbot/data/Training/data"
    # zipfile 객체 생성
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # 추출
        zip_ref.extractall(extract_path)

In [36]:
# 읽어올 파일들이 들어있는 폴더 경로
folder_path = "C:/dev/github/pythonbasic/mental-health-chatbot/data/Training/data"

# 폴더 내의 모든 txt 파일 경로 리스트 생성
file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.txt')]

# 각 파일을 읽어와서 DataFrame으로 합치기
i = 0
df_list = []
for file_path in file_paths:
    with open(file_path, 'r', encoding = "utf-8") as f:
        i += 1

        # 각 파일의 내용을 DataFrame으로 변환
        df = pd.read_csv(f, sep='\t', header=None)
        df['구분'] = i 
        df_list.append(df)

# 모든 DataFrame 합치기
result_df = pd.concat(df_list, ignore_index= False)

In [61]:
result_df

,0,구분
0,1 : 애덜앙 나 너무 배불러서 배 아파,1
1,2 : 배 아프면 약국 가서 소화제라도 마시는 거 어때,1
2,3 : 얼마나 먹었다고 배가 아프데,1
3,1 : 떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,1
4,2 : 하긴 *이 배는 파리보다 작으니까 당연히 그만큼 먹어도 배부르지,1
...,...,...
11,2 : 그러니깐 ㅠ 이제 해리 자식들로 시작해야지,71651
12,1 : 근데 해리만큼 잘 할 수 있을까?,71651
13,2 : 진짜 그 3인방 케미가 최고였는데,71651
14,1 : 그치그치 완전 최고였는데 ㅜㅠ,71651


라벨링 데이터 (.json)

In [56]:
# 추출할 zip 파일 경로
for i in range(1,5):
    zip_file_path = f"C:/dev/github/pythonbasic/mental-health-chatbot/data/Training/TL_01. KAKAO({i}).zip"

    # 추출할 위치
    extract_path = "C:/dev/github/pythonbasic/mental-health-chatbot/data/Training/data_labeling"

    # zipfile 객체 생성
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # 추출
        zip_ref.extractall(extract_path)
    

In [57]:
import zipfile

# 추출할 zip 파일 경로
zip_file_path = "C:/dev/github/pythonbasic/mental-health-chatbot/data/Validation/VL_01. KAKAO.zip"

# 추출할 위치
extract_path = "C:/dev/github/pythonbasic/mental-health-chatbot/data/Validation/VL_01. KAKAO"

# zipfile 객체 생성
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:

    # 추출
    zip_ref.extractall(extract_path)

In [58]:
# 읽어올 파일들이 들어있는 폴더 경로
folder_path = "C:/dev/github/pythonbasic/mental-health-chatbot/data/Training/data_labeling"

# 폴더 내의 모든 txt 파일 경로 리스트 생성
file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.json')]

# 각 파일을 읽어와서 DataFrame으로 합치기
i = 0
df_list = []
for file_path in file_paths:
    with open(file_path, 'r', encoding = "utf-8") as f:
        i += 1

        # 각 파일의 내용을 DataFrame으로 변환
        df = pd.read_csv(f, sep='\t', header=None)
        df['구분'] = i 
        df_list.append(df)

# 모든 DataFrame 합치기
result_df_json = pd.concat(df_list, ignore_index= False)

In [62]:
result_df_json

,0,구분
0,{,1
1,"""dataset"": {",1
2,"""identifier"": 71459,",1
3,"""name"": ""KAKAO_1000_01_set"",",1
4,"""src_path"": ""/data/file/cubeManager/PROJEC...",1
...,...,...
222,],71651
223,},71651
224,},71651
225,],71651


In [60]:
result_df_json[0][24]

24                 "subject": "식음료",
24              "subject": "주거와 생활",
24                  "subject": "교통",
24            "subject": "회사/아르바이트",
24                  "subject": "군대",
                   ...              
24                  "subject": "미용",
24                  "subject": "건강",
24              "subject": "상거래 전반",
24               "subject": "방송/연예",
24               "subject": "영화/만화",
Name: 0, Length: 71651, dtype: object

In [75]:
result_df_json[0][24].values

array(['        "subject": "식음료",', '        "subject": "주거와 생활",',
       '        "subject": "교통",', ..., '        "subject": "상거래 전반",',
       '        "subject": "방송/연예",', '        "subject": "영화/만화",'],
      dtype=object)

# 전처리

In [68]:
result_df

,0,구분
0,1 : 애덜앙 나 너무 배불러서 배 아파,1
1,2 : 배 아프면 약국 가서 소화제라도 마시는 거 어때,1
2,3 : 얼마나 먹었다고 배가 아프데,1
3,1 : 떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,1
4,2 : 하긴 *이 배는 파리보다 작으니까 당연히 그만큼 먹어도 배부르지,1
...,...,...
11,2 : 그러니깐 ㅠ 이제 해리 자식들로 시작해야지,71651
12,1 : 근데 해리만큼 잘 할 수 있을까?,71651
13,2 : 진짜 그 3인방 케미가 최고였는데,71651
14,1 : 그치그치 완전 최고였는데 ㅜㅠ,71651


In [64]:
result_df_json[0][24]

24                 "subject": "식음료",
24              "subject": "주거와 생활",
24                  "subject": "교통",
24            "subject": "회사/아르바이트",
24                  "subject": "군대",
                   ...              
24                  "subject": "미용",
24                  "subject": "건강",
24              "subject": "상거래 전반",
24               "subject": "방송/연예",
24               "subject": "영화/만화",
Name: 0, Length: 71651, dtype: object

In [72]:
for group, data in result_df.groupby('구분'):
    s = result_df_json[0][24]
    result_df.extend(s)

result_df['subject'] = result_df

AttributeError: 'DataFrame' object has no attribute 'extend'